In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd

PATH = 'chromedriver'

In [2]:
driver = webdriver.Chrome(PATH)

# Function Solution

In [52]:
#return the df
def get_data(url, findId, headerIndex):
    driver.get(url)
    res = driver.find_element(By.ID, findId)
    rows = res.find_elements(By.TAG_NAME, "tr")
    #get column names from first row
    cols = []
    for ele in rows[headerIndex].find_elements(By.TAG_NAME, "th"):
        cols.append(ele.text)
    cols[1:]
    #create dataframe and append table rows to it
    stats = pd.DataFrame(columns = cols[1:])
    for row in rows:
        player = []
        for td in row.find_elements(By.TAG_NAME, "td"):
            player.append(td.text)
        #try except is needed here so that header rows don't throw an error when added to the df and are instead skipped
        try:
            stats.loc[len(stats)] = player
        except:
            continue
    return(stats)

In [6]:
gsc_stats  = get_data('https://fbref.com/en/comps/22/gca/Major-League-Soccer-Stats', 'stats_gca', 1)
defense_stats = get_data('https://fbref.com/en/comps/22/defense/Major-League-Soccer-Stats', 'stats_defense', 1)
keeper_stats = get_data('https://fbref.com/en/comps/22/keepers/Major-League-Soccer-Stats', 'stats_keeper', 1)
misc_stats = get_data('https://fbref.com/en/comps/22/misc/Major-League-Soccer-Stats', 'stats_misc', 1)
pass_stats = get_data('https://fbref.com/en/comps/22/passing/Major-League-Soccer-Stats', 'stats_passing', 1)
possession_stats = get_data('https://fbref.com/en/comps/22/possession/Major-League-Soccer-Stats', 'stats_possession', 1)
shooting_stats = get_data('https://fbref.com/en/comps/22/shooting/Major-League-Soccer-Stats', 'stats_shooting', 1)
time_stats = get_data('https://fbref.com/en/comps/22/playingtime/Major-League-Soccer-Stats', 'stats_playing_time', 1)

In [53]:
east_squads = get_data('https://fbref.com/en/comps/22/Major-League-Soccer-Stats#all_stats_squads_standard', 'results114991Eastern-Conference_overall', 0)
west_squads = get_data('https://fbref.com/en/comps/22/Major-League-Soccer-Stats#all_stats_squads_standard', 'results114991Western-Conference_overall', 0)

squad_goalkeeping = get_data('https://fbref.com/en/comps/22/keepers/Major-League-Soccer-Stats', 'stats_squads_keeper_for', 1)
squad_gsc  = get_data('https://fbref.com/en/comps/22/gca/Major-League-Soccer-Stats', 'stats_squads_gca_for', 1)
squad_defense = get_data('https://fbref.com/en/comps/22/defense/Major-League-Soccer-Stats', 'stats_squads_defense_for', 1)
squad_misc = get_data('https://fbref.com/en/comps/22/misc/Major-League-Soccer-Stats', 'stats_squads_misc_for', 1)
squad_pass = get_data('https://fbref.com/en/comps/22/passing/Major-League-Soccer-Stats', 'stats_squads_passing_for', 1)
squad_possession = get_data('https://fbref.com/en/comps/22/possession/Major-League-Soccer-Stats', 'stats_squads_possession_for', 1)
squad_shooting = get_data('https://fbref.com/en/comps/22/shooting/Major-League-Soccer-Stats', 'stats_squads_shooting_for', 1)
squad_time = get_data('https://fbref.com/en/comps/22/playingtime/Major-League-Soccer-Stats', 'stats_squads_playing_time_for', 1)


In [37]:
#create function to add squad name to each squad df
squad_dict = {0:'Atlanta Utd', 1: 'Austin FC', 2:'CF Montréal', 3:'Charlotte FC', 4: 'Chicago Fire', 5:'Colorado Rapids', 6:'Columbus Crew', 7:'D.C. United', 8:'FC Cincinnati', 9:'FC Dallas', 10:'Houston Dynamo', 11:'Inter Miami', 12:'LA Galaxy', 13:'Los Angeles FC', 14:'Minnesota Utd', 15:'Nashville', 16:'New England', 17:'NY Red Bulls', 18:'NYCFC', 19:'Orlando City', 20:'Philadelphia', 21:'Portland Timbers', 22:'Real Salt Lake', 23:'San Jose', 24:'Seattle', 25:'Sporting KC', 26:'Toronto FC', 27:'Vancouver'}

def add_squads(df):
    df.reset_index(inplace = True)
    df['squad'] = df['index'].apply(lambda x: squad_dict[x])
    df.drop('index', inplace = True, axis = 1)

In [39]:
add_squads(squad_goalkeeping)
add_squads(squad_gsc)
add_squads(squad_defense)
add_squads(squad_misc)
add_squads(squad_pass)
add_squads(squad_possession)
add_squads(squad_shooting)
add_squads(squad_time)

In [56]:
squads = pd.concat([east_squads, west_squads]).reset_index(drop=True)

# Database prep and creation

In [64]:
defense_stats.reset_index()

player_ids = dict(zip(defense_stats.Player, defense_stats.index))

In [65]:
player_ids
#add ids to each player df

{'Paxten Aaronson': 0,
 'Lalas Abubakar': 1,
 'Nicolás Acevedo': 2,
 'Ifunanyachi Achara': 3,
 'Bryan Acosta': 4,
 'George Acosta': 5,
 'Kellyn Acosta': 6,
 'Luciano Acosta': 7,
 'Mo Adams': 8,
 'Samuel Adeniran': 9,
 'Harrison Afful': 10,
 'William Agada': 11,
 'Oskar Ågren': 12,
 'Daniel Aguirre': 13,
 'Ali Ahmed': 14,
 'Arnaud Ake': 15,
 'Tesho Akindele': 16,
 'Ayo Akinola': 17,
 'Jordy Alcivar': 18,
 'Caio Alexandre': 19,
 'Tony Alfaro': 20,
 'Noah Allen': 21,
 'Thiago Almada': 22,
 'Osvaldo Alonso': 23,
 'Jozy Altidore': 24,
 'Efrain Alvarez': 25,
 'Luis Amarilla': 26,
 'Frankie Amaya': 27,
 'Mikey Ambrose': 28,
 'Malte Amundsen': 29,
 'Max Anchor': 30,
 'Sebastian Anderson': 31,
 'José Andrés Martínez': 32,
 'Jalil Anibaba': 33,
 'Themi Antonoglou': 34,
 'Brian Anunga': 35,
 'Cristian Arango': 36,
 'César Araujo': 37,
 'Julian Araujo': 38,
 'Luiz Araújo': 39,
 'Daniel Armando Ríos': 40,
 'Adam Armour': 41,
 'Xavier Arreaga': 42,
 'Kervin Arriaga': 43,
 'Paul Arriola': 44,
 'Artur